In [1]:
!python --version

Python 3.7.13


In [2]:
!pip install --upgrade snowflake-connector-python

     |████████████████████████████████| 24.5 MB 20.5 MB/s eta 0:00:01
     |████████████████████████████████| 105 kB 109.4 MB/s eta 0:00:01
     |████████████████████████████████| 194 kB 105.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 92.9 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow-cml-plugin 0.0.1 requires typing-extensions==4.1.1, but you have typing-extensions 4.6.3 which is incompatible.


In [4]:
#checking version of Linux Distribution and version
!cat /etc/*-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04.6 LTS"
NAME="Ubuntu"
VERSION="20.04.6 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.6 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


In [1]:
#check if OPEN SSL already exists or we do pip install. 
import ssl
print(ssl.OPENSSL_VERSION)

OpenSSL 1.1.1f  31 Mar 2020


In [12]:
#!/usr/bin/env python
import snowflake.connector
import os

##format
# conn = snowflake.connector.connect(
#     user='XXXX',
#     password='XXXX',
#     account='XXXX',
#     session_parameters={
#         'QUERY_TAG': 'SnowflakeConnectorDemo',
#     }
# )
# Gets the version
ctx = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USERNAME"),
    password=os.getenv("SNOWFLAKE_PWD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT")
    )
cs = ctx.cursor()
try:
    cs.execute("SELECT current_version()")
    one_row = cs.fetchone()
    print(one_row[0])
finally:
    cs.close()
ctx.close()

7.21.1


In [13]:

#set Query Parameters. 
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USERNAME"),
    password=os.getenv("SNOWFLAKE_PWD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    session_parameters={
        'QUERY_TAG': 'SnowflakeConnectorDemo',
    } 
    )

In [14]:
cursor_list = conn.cursor().execute("SHOW WAREHOUSES")

In [15]:
conn.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")

In [16]:
#use Cursors to iterate through warehouses
for cursor in cursor_list:
      print(cursor[0])

COMPUTE_WH
TINY_WAREHOUSE_MG


In [17]:
 conn.cursor().execute("USE WAREHOUSE COMPUTE_WH")

In [18]:
conn.cursor().execute("USE DATABASE SNOWFLAKE_SAMPLE_DATA")

In [19]:
#use the schema
conn.cursor().execute("USE SCHEMA TPCH_SF1")

In [20]:
#single row info
#col1, col2 = conn.cursor().execute("SELECT C_CUSTKEY, C_NAME, C_NATIONKEY, C_PHONE, C_ACCTBAL, C_ADDRESS FROM CUSTOMER").fetchone()
col1, col2 = conn.cursor().execute("SELECT C_CUSTKEY, C_NAME FROM CUSTOMER").fetchone()

print('{0}, {1}'.format(col1, col2))

30001, Customer#000030001


In [13]:
#import Snowflake Pandas interface
!pip install "snowflake-connector-python[pandas]"

     |████████████████████████████████| 36.0 MB 22.2 MB/s eta 0:00:01


In [21]:
#fetch into a Pandas DF
import pandas as pd
cur = conn.cursor().execute("SELECT C_CUSTKEY, C_NAME, C_NATIONKEY, C_PHONE, C_ACCTBAL, C_ADDRESS FROM CUSTOMER LIMIT 10")
#pd = cur.fetch_pandas_all()
try: 
    all_rows = cur.fetchall()
    num_fields = len(cur.description)
    field_names = [i[0] for i in cur.description]
finally:
    cur.close()

conn.close()

df = pd.DataFrame(all_rows)
df.columns = field_names

In [22]:
df.head()

,C_CUSTKEY,C_NAME,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_ADDRESS
0,60001,Customer#000060001,14,24-678-784-9652,9957.56,9Ii4zQn9cX
1,60002,Customer#000060002,15,25-782-500-8435,742.46,ThGBMjDwKzkoOxhz
2,60003,Customer#000060003,16,26-859-847-7640,2526.92,"Ed hbPtTXMTAsgGhCr4HuTzK,Md2"
3,60004,Customer#000060004,10,20-573-674-7999,7975.22,"NivCT2RVaavl,yUnKwBjDyMvB42WayXCnky"
4,60005,Customer#000060005,12,22-741-208-1316,2504.74,"1F3KM3ccEXEtI, B22XmCMOWJMl"
